In [201]:
from bs4 import BeautifulSoup
import requests
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from medscraper import medsearch

In [202]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')

In [203]:
url="https://www.medicinpriser.dk/default.aspx?lng=2"

In [204]:
driver=webdriver.Chrome(options=options)
driver.get(url)
driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").send_keys("nepafenac")
driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)

In [205]:
#Identificar la tabla que cuenta con estas características "<table cellpadding="0" border="0" cellspacing="0" style="width:100%">

table_element = driver.find_element(By.XPATH, "//table[@cellpadding='0' and @border='0' and @cellspacing='0' and @style='width:100%']")
table_contents = table_element.get_attribute('outerHTML')
soup = BeautifulSoup(table_contents, 'html.parser')

In [206]:
producto=[]
presentacion=[]
precio=[]
for i in soup.find_all('div', {'class':'width282'}):
    j=i.find_all('table', {'class':'grid'})


In [207]:
tableprod = soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridStatic'})
tableprecio=soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridScroll'})


In [208]:
#obtener rows de las dos tablas
rowsprod = tableprod.find_all('tr')
rowsprecio=tableprecio.find_all('tr')

In [209]:
# And then we can loop through the rows to extract the data
caract=[]

for row in rowsprod:
    columns = row.find_all('td')
    for column in columns:
        #Append el texto con saltos de 2 en 2
        caract.append(column.text)

producto=caract[::3]
producto=[re.sub(r'\d+', '', i) for i in producto]
presentacion=caract[1::3]
paquete=caract[2::3]

caract2=[]
for row in rowsprecio:
    columns = row.find_all('td')
    for column in columns:
        #Append el texto con saltos de 2 en 2
        caract2.append(column.text)

#Obtener precio en múltiplos de 5
precio=caract2[4::5]
#Sustituir coma por punto
precio=[re.sub(r',', '.', i) for i in precio]


In [210]:
def euro():

    exchangerates='https://openexchangerates.org/api/latest.json?app_id=52adcacdba9c48ce8430667723c02968&base=USD&symbols=MXN,EUR'
    r=requests.get(exchangerates).json()
    #Obtener tipos de cambio
    euro=r['rates']['MXN']/r['rates']['EUR']
    return euro


In [211]:
df=pd.DataFrame({'producto':producto, 'presenta':presentacion, 'paquete':paquete, 'precio':precio})
df[['presentacion','cantidad']]=df.presenta.str.split(" ",expand=True)
df=df[~df.precio.str.contains("Discont")]
df["precio"]=df["precio"].astype(float)
df["precio"]=round(df["precio"]*euro(),2)
df

,producto,presenta,paquete,precio,presentacion,cantidad
0,Nevanac,1 mg/ml,"5 mløjendråber, susp.",501.56,1,mg/ml
1,Nevanac,1 mg/ml,"5 ml (Orifarm)øjendråber, susp.",495.56,1,mg/ml
2,Nevanac,1 mg/ml,"5 ml (2care4)øjendråber, susp.",498.00,1,mg/ml
3,Nevanac,3 mg/ml,"3 ml (2care4)øjendråber, susp.",1001.24,3,mg/ml
5,Nevanac,3 mg/ml,"3 ml (Orifarm)øjendråber, susp.",998.80,3,mg/ml


In [214]:
s=medsearch.scraper("fsanpablo","nevanac")
mex=s.show_dataframe()
mex

Realizando la búsqueda en Farmacias San Pablo de:  nevanac


,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod
0,nevanac 5 ml suspension gotero nepafenaco 1 mg,641.0,19:03:56 23/06/2023,fsanpablo,nevanac,farmacias san pablo,Mexico,nevanac,5.0,ml,suspension,1


In [215]:
mex["precio"]-df["precio"][0]

0    139.44
Name: precio, dtype: float64